In [1]:
from utils.config_loader import load_config, assign_config
from processing.climatology import process_climatology
from processing.preprocessing import preprocess_data
from processing.normalize import normalize_data
from model.train import train_model
from model.inference import infer
from model.metrics import calculate_rmse
from visualization.plots import plot_heatmaps

config = load_config("./config/params.yaml")

Login to wandb and init experiment

In [2]:
import wandb
import os

os.environ['WANDB_NOTEBOOK_NAME'] = "pipeline.ipynb"
wandb.login()

run = wandb.init(
    project="WindViVit2",
    
    notes="Testing reconstruct everything",
    tags=["8 years data", "LinearLR", "BERT"]
)

wandb: Currently logged in as: samuelreyes. Use `wandb login --relogin` to force relogin


Assign config to wandb experiment

In [3]:

wandb_config = wandb.config

assign_config(wandb_config, config)

### Data preprocessing

In [2]:
process_climatology(config)

In [3]:
preprocess_data(config)

In [4]:
normalize_data(config)

### Training

In [4]:
train_model(config)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.53 GiB (GPU 0; 23.69 GiB total capacity; 20.49 GiB already allocated; 1.74 GiB free; 21.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Evaluate model

In [ ]:
results = infer(config)

In [ ]:
rmse = calculate_rmse(results)
wandb.log({"rmse": rmse})
rmse

In [ ]:
fig = plot_heatmaps(config, results, 1500, 5)

In [ ]:
wandb.log({"heatmap": fig})

In [ ]:
run.finish()